### Preparation

In [15]:
from elasticsearch import Elasticsearch, helpers
from nlp_common.acts_reader import ActsReader
import json

INDEX_NAME = 'polish-bills'

In [4]:
def create_index(client: Elasticsearch):
  if client.indices.exists(INDEX_NAME):
    print(f'Index {INDEX_NAME} already exists. Skipping...')
    return
  
  print(f'Index {INDEX_NAME} doesn\'t exists. Creating...')
  client.indices.create(
      index=INDEX_NAME,
      body={
        'settings': {
          'number_of_shards': 1,
          'number_of_replicas': 0,
          'analysis': {
            'analyzer': {
              'polish-bills-analyzer': {
                'type': 'custom',
                'tokenizer': 'standard',
                'char_filter': [
                  'kodeks_synonyms'
                ],
                'filter': [
                  'morfologik_stem',
                  'lowercase'
                ]
              }
            },
            'char_filter': {
              'kodeks_synonyms': { 
                'type': 'mapping',
                'mappings': [
                  'kpk => kodeks postępowania karnego',
                  'kpc => kodeks postępowania cywilnego',
                  'kk => kodeks karny',
                  'kc => kodeks cywilny'
                ]
              }
            },
          }
        }
      },
    )


def index_bills(client: Elasticsearch):
  if client.indices.stats(INDEX_NAME)['_all']['primaries']['docs']['count'] > 0:
    print('Some docs were already indexed. Skipping...')
    return  
  
  print(f'Indexing bills...')
  reader = ActsReader('../ustawy')
  sources = [ {
    '_index': INDEX_NAME,
    '_source': {
      'name': name, 
      'text': bill
    }
  } for name, _, bill in reader.all_acts() ]
  
  helpers.bulk(client, sources)
    
  print('Indexing completed.')
  

In [5]:
client = Elasticsearch("http://localhost:9200")

resp = client.info()

print("Elastic info")
print(resp)

create_index(client)
index_bills(client)  

Elastic info
{'name': '85ef8b70e41e', 'cluster_name': 'docker-cluster', 'cluster_uuid': '_woZQSdVTj6ABS-nYRXEgg', 'version': {'number': '7.10.1', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '1c34507e66d7db1211f66f3513706fdf548736aa', 'build_date': '2020-12-05T01:00:33.671820Z', 'build_snapshot': False, 'lucene_version': '8.7.0', 'minimum_wire_compatibility_version': '6.8.0', 'minimum_index_compatibility_version': '6.0.0-beta1'}, 'tagline': 'You Know, for Search'}
Index polish-bills already exists. Skipping...
Some docs were already indexed. Skipping...


### Number of acts containing word 'ustawa'

### Most relevant docs for word 'konstytucja'

In [30]:
response = client.search(index=INDEX_NAME, body={'query': {'match': { 'text': 'konstytucja' }}})
print(response['h)
for hit in response['hits']:
    #print(json.loads(hit)['source']['name'])
    pass

{'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 2, 'relation': 'eq'}, 'max_score': 9.633655, 'hits': [{'_index': 'polish-bills', '_type': '_doc', '_id': '8y8GznwBoO7ZI9TtemPt', '_score': 9.633655, '_source': {'name': '1997_629.txt', 'text': '\n\n\n\nDz.U. z 1994 r. Nr 61, poz. 251\n                         USTAWA  KONSTYTUCYJNA\n                       z dnia 22 kwietnia 1994 r.\n                                    \n         o zmianie ustawy konstytucyjnej o trybie przygotowania\n           i uchwalenia Konstytucji Rzeczypospolitej Polskiej\n                                    \n                                    \n                                Art. 1.\nW ustawie  konstytucyjnej z  dnia 23 kwietnia 1992 r. o trybie przygotowania i \nuchwalenia Konstytucji Rzeczypospolitej Polskiej (Dz.U. Nr 67, poz. 336) \nwprowadza się następujące zmiany:\n \n 1) po art. 2 dodaje się art. 2a, art. 2b i art. 2c w brzmie